## Web Scraper Wallapop

Initialize Libraries

In [8]:
#We import all the relevant libraries for the project
#first we import all of the selenium functions we need for scraping
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options

#we now import time to create sleeps to allow the page to load, numpy to create NAN's and Pandas
import time
import numpy as np
import pandas as pd
import re
from datetime import datetime

## Web Scraping

We initialize the Chrome driver

In [9]:
#we make the page fullscreen to ensure the dynamic components are shown
#chrome_options = Options()
#chrome_options.add_argument("--headless")
driver = webdriver.Chrome()
driver.maximize_window()

Go to the main page and accept cookies

In [10]:
# go to the main page and accept cookies if applicable

base_URL = 'https://es.wallapop.com'

driver.get(base_URL)

try: 
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))).click()
except:
    pass

Search for bicicletas and Apply the general filters

In [11]:
#go into the searchbox and find bicicletas

WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Buscar en Todas las categorías']")))
searchbox = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Buscar en Todas las categorías']")))
time.sleep(1)
searchbox.clear()
time.sleep(1)
searchbox.send_keys("bicicletas")
time.sleep(3)
WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.Search__suggestion-title--highlighted'))).click()

time.sleep(2)

#Look only for barcelona
WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-bubble/div'))).click()
location_searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/div[1]/input")))
location_searchbox.clear()
location_searchbox.send_keys('Barcelona')
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/div[1]/ngb-typeahead-window/button[1]/div/span"))).click()
time.sleep(3)
slider = driver.find_element(By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-slider-form/form/ngx-slider/span[4]')
knob=slider.find_element(By.XPATH,'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-slider-form/form/ngx-slider/span[5]')
move=webdriver.ActionChains(driver)
move.click_and_hold(knob).move_by_offset(slider.size['width']*-0.65,0).release().perform()
button_to_click = driver.find_elements(By.XPATH, "//button[contains(text(),'Aplicar')]")[0]
WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button_to_click)).click()

time.sleep(2)

# set the price to 800€
WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//div[contains(text(),"Precio")]'))).click()
location_searchbox = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//input[@placeholder="Sin límite"]')))
location_searchbox.clear()
location_searchbox.send_keys('800')
time.sleep(2)
button_to_click = driver.find_elements(By.XPATH, "//button[contains(text(),'Aplicar')]")[1]
WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button_to_click)).click()

time.sleep(2)


In [12]:
# Spanish mappings used to parse the dates
months = {
    'ene': '01',
    'feb': '02',
    'mar': '03',
    'abr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dic': '12'
}


# Bonus
def parse_date(date):
    try:
        split = date.split('-')
        split[1] = months[split[1]]

        return datetime.strptime('-'.join(split), '%d-%m-%Y')
    except:
        return np.NAN

Get the elements to interpret into the DF

In [13]:
titles = []
prices = []
descriptions =[]
post_urls =[]
img_urls = []
types = []
post_dates =[]
states=[]
children = []
bike_size_letter = []
bike_size_number = []

In [14]:
#We create iterable lists for the filters we want to combine
iterable_types = ['MTB', 'Bicicletas plegables', 'Bicicletas de carretera']
iterable_states = ['Nuevo','Como nuevo','En buen estado']


try:
    driver.find_element(By.XPATH, "//button[contains(text(),'Ver más productos')]").click()
except: 
    pass

for i in iterable_types:
    WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//div[contains(text(),"Subcategoría")]'))).click()
    time.sleep(0.5)
    WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//p[contains(text(),"Bicicletas y triciclos")]'))).click()
    time.sleep(0.5)
    
    elements  = driver.find_elements(By.CSS_SELECTOR,".MultiSelectOption")

    for items in elements:
    
        if items.text == i:
            items.find_element(By.CSS_SELECTOR,'.Checkbox__mark').click()


    button_to_click = driver.find_elements(By.XPATH, "//button[contains(text(),'Aplicar')]")[2]
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button_to_click)).click()


    for j in iterable_states:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,'//div[contains(text(),"Estado del producto")]'))).click()
        time.sleep(0.5)

        try:
            driver.find_element(By.XPATH, "//button[contains(text(),'Ver más productos')]").click()
        except: 
            pass

        elements  = driver.find_elements(By.CSS_SELECTOR,".MultiSelectOption")

        for items in elements:
            if items.find_element(By.CSS_SELECTOR,'.MultiSelectOption__label') != '':
                if items.find_element(By.CSS_SELECTOR,'.MultiSelectOption__label').text == j :
                    items.find_element(By.CSS_SELECTOR,'.Checkbox__mark').click()
        time.sleep(0.5)

        button_to_click = driver.find_elements(By.XPATH, "//button[contains(text(),'Aplicar')]")[3]
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button_to_click)).click()
        
        time.sleep(1.5)
        driver.execute_script("window.scrollBy(0,1000)")

        comb_elements = []
        scrolls = 0

        while len(comb_elements) < 250: 

            driver.execute_script("window.scrollBy(0,1000)")
            #print(scrolls)
            scrolls += 1
            comb_elements  = driver.find_elements(By.CSS_SELECTOR,".ItemCardList__item")
            time.sleep(0.1)
            if scrolls > 100:
                break

        if len(comb_elements)>250:
            comb_elements = comb_elements[:250]

        print(i,j)
        print('Scrolls: ',scrolls)
        print('Elements: ',len(comb_elements))

        
        for items in comb_elements:

            #append the title
            try:
                if items.find_element(By.CSS_SELECTOR,'.ItemCard__title').text:
                    bike_title = items.find_element(By.CSS_SELECTOR,'.ItemCard__title').text
                    titles.append(bike_title)
            except:
                titles.append(np.NAN)

            #append the price
            try:
                price = items.find_element(By.CSS_SELECTOR,'.ItemCard__price').text.split('€')[0]
                prices.apend(float(price))
            except:
                prices.append(np.NAN)

            #append the iamge url
            try:
                img_urls.append(items.find_element(By.TAG_NAME,'img').get_attribute('src'))
            except:
                img_urls.append(np.NAN)

            # we have all the information needed from this page so we can move on to the post page
            # the description sometimes doesn't load on the main page so we extract it from the post

            items.click()
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(1.5)

            #append the type
            try:
                types.append(driver.find_elements(By.CSS_SELECTOR,'.ExtraInfo__taxonomyBubble')[1].text)
            except:
                types.append(np.NAN)

            #append the post date
            try:
                date = driver.find_element(By.CSS_SELECTOR,'.card-product-detail-user-stats-published').text
                post_dates.append(parse_date(date))
            except:
                post_dates.append(np.NAN)    

            #append the state
            try:
                states.append(driver.find_element(By.CSS_SELECTOR,'.ExtraInfo__text').text)
            except:
                states.append(np.NAN)

            #append the description
            try:
                bike_description = driver.find_element(By.CSS_SELECTOR,'.js__card-product-detail--description').text
                descriptions.append(bike_description)
            except:
                descriptions.append(np.NAN)

            #append the children

            # This section checks if the bike is for children or not (having any of the mentions: “niño/a”, “niño”, “niña”, “niños” , “niñas” or "niño/as")
            try:

                if ("niño/a" or "niño" or "niña" or "niños" or "niñas" or "niño/as") in bike_title.lower():
                    children.append(True)
                elif ("niño/a" or "niño" or "niña" or "niños" or "niñas" or "niño/as") in bike_description.lower():
                    children.append(True)
                else:
                    children.append(False)
            except:
                children.append(False)
        
            #append letter size

            try:

                if re.findall(" s ", bike_description, flags=re.IGNORECASE):    
                    bike_size_letter.append("S")
                elif re.findall(" m ", bike_description, flags=re.IGNORECASE):
                    bike_size_letter.append("M")
                elif re.findall(" l ", bike_description , flags=re.IGNORECASE):
                    bike_size_letter.append("L")
                elif re.findall("M/L", bike_description, flags=re.IGNORECASE):
                    bike_size_letter.append("M/L")
                else:
                    bike_size_letter.append(np.nan)
            except:
                bike_size_letter.append(np.nan)


            try:
                    # This piece of code takes the numerical size value for each listing (if it exists) and appends it to the dataframe
                if " talla " in bike_description.lower():
                    
                    talla_index = bike_description.lower().split().index('talla')
                    mini_description = bike_description.split()[talla_index : talla_index + 3]
                    new_description_small = ' '.join(mini_description)
                    #int_in_new_description_small = re.findall(r'\d+', new_description_small)
                    
                    # The typical range of bike sizes is around 50 to 60 and thus this range will be used
                    if '50' in new_description_small:
                        bike_size_number.append(int(50))
                    elif '51' in new_description_small:
                        bike_size_number.append(int(51))
                    elif '52' in new_description_small:
                        bike_size_number.append(int(52))
                    elif '53' in new_description_small:
                        bike_size_number.append(int(53))
                    elif '54' in new_description_small:
                        bike_size_number.append(int(54))
                    elif '55' in new_description_small:
                        bike_size_number.append(int(55))
                    elif '56' in new_description_small:
                        bike_size_number.append(int(56))
                    elif '57' in new_description_small:
                        bike_size_number.append(int(57))
                    elif '58' in new_description_small:
                        bike_size_number.append(int(58))
                    elif '59' in new_description_small:
                        bike_size_number.append(int(59))
                    elif '60' in new_description_small:
                        bike_size_number.append(int(60))
                    else:
                        bike_size_number.append(np.nan)
                
                else:
                    bike_size_number.append(np.nan)
                
            except:
                bike_size_number.append(np.nan)

            time.sleep(0.1)


            #append the post url by getting the driver url before closing and moving to the next item
            post_urls.append(driver.current_url)

            time.sleep(0.1)
            driver.close()  
            driver.switch_to.window(driver.window_handles[0])

        

        driver.find_elements(By.CSS_SELECTOR,'.Bubble__clear')[2].click()
        time.sleep(1)

        
    driver.find_elements(By.CSS_SELECTOR,'.Bubble__clear')[1].click()
    time.sleep(2)
   

MTB Nuevo
Scrolls:  101
Elements:  74
MTB Como nuevo
Scrolls:  60
Elements:  250
MTB En buen estado
Scrolls:  54
Elements:  250
Bicicletas plegables Nuevo
Scrolls:  101
Elements:  48
Bicicletas plegables Como nuevo
Scrolls:  101
Elements:  157
Bicicletas plegables En buen estado
Scrolls:  101
Elements:  140
Bicicletas de carretera Nuevo
Scrolls:  101
Elements:  51
Bicicletas de carretera Como nuevo
Scrolls:  101
Elements:  206
Bicicletas de carretera En buen estado
Scrolls:  39
Elements:  250


In [15]:
df = pd.DataFrame({'Title':titles,
                    'Description':descriptions,
                    'Price':prices,
                    'Image URL': img_urls,
                    'Post URL':post_urls,
                    'State':states,
                    'Post Date': post_dates,
                    'Type':types,
                    'Children':children,
                    'Letter Size':bike_size_letter,
                    'Number Size':bike_size_number
                    })


In [16]:
driver.close()

In [17]:
df

,Title,Description,Price,Image URL,Post URL,State,Post Date,Type,Children,Letter Size,Number Size
0,Bicicleta Specialized HardRock como nueva,"Specialized HardRock, 100% aluminio. Ruedas 26...",NaN,https://cdn.wallapop.com/images/10420/e5/vi/__...,https://es.wallapop.com/item/bicicleta-special...,Nuevo,2022-12-09,MTB,False,NaN,NaN
1,bicicleta megamo,"Bicicleta NUEVA a estrenar, talla S, rueda 26,...",NaN,https://cdn.wallapop.com/images/10420/e5/kh/__...,https://es.wallapop.com/item/bicicleta-megamo-...,Nuevo,2022-12-07,MTB,False,NaN,NaN
2,computadora para bicicleta a estrenar,Computadora para bicicleta a estrenar con sus ...,NaN,https://cdn.wallapop.com/images/10420/e4/ke/__...,https://es.wallapop.com/item/computadora-para-...,Nuevo,2022-12-09,MTB,False,NaN,NaN
3,Bici b-pro,Vendo MTB comprada en 2022. Esta como nueva mu...,NaN,https://cdn.wallapop.com/images/10420/e4/30/__...,https://es.wallapop.com/item/bici-b-pro-853385713,Nuevo,2022-12-07,MTB,False,NaN,NaN
4,Bicicleta mtb lady 26,Bicicleta denbike mtb 26”. Es nueva a estrenar...,NaN,https://cdn.wallapop.com/images/10420/e3/qg/__...,https://es.wallapop.com/item/bicicleta-mtb-lad...,Nuevo,2022-11-27,MTB,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1421,Giant TCR Talla 50. Aluminio-Carbono,Bicicleta cuadro de aluminio y componentes de ...,NaN,https://cdn.wallapop.com/images/10420/e1/jq/__...,https://es.wallapop.com/item/giant-tcr-talla-5...,Bueno,2022-11-15,Bicicletas de carretera,False,NaN,NaN
1422,Vitus 992 años'90,Bicicleta de carretera personalizada de los añ...,NaN,https://cdn.wallapop.com/images/10420/e0/0o/__...,https://es.wallapop.com/item/bicicleta-de-carr...,Bueno,2022-12-01,Bicicletas de carretera,False,NaN,50.0
1423,kross talla M,"Bicicleta kross, talla m. la tengo en venta po...",NaN,https://cdn.wallapop.com/images/10420/e0/g2/__...,https://es.wallapop.com/item/kross-talla-m-847...,Bueno,2022-11-27,Bicicletas de carretera,False,NaN,NaN
1424,Bicicletta,Muy buenas Vendo bicicleta usada Lo único que ...,NaN,https://cdn.wallapop.com/images/10420/dy/px/__...,https://es.wallapop.com/item/bicicletta-844376638,Bueno,2022-11-01,Bicicletas de carretera,False,NaN,NaN


In [18]:
df.dtypes

Title                  object
Description            object
Price                 float64
Image URL              object
Post URL               object
State                  object
Post Date      datetime64[ns]
Type                   object
Children                 bool
Letter Size            object
Number Size           float64
dtype: object